In [347]:
from os import path, getcwd
from IPython.display import Markdown as md
import pandas as pd
import numpy as np
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import patsy

strYearDBPath=path.join('..','DB', 'year.sqlite3') # путь к рабочей базе данных фактических рядов SQLite 
strExYearDBPath=path.join('..','DB', 'exog_year.sqlite3') # путь к рабочей базе данных экзогенных переменных SQLite
strExParamDBPath=path.join('..','DB', 'exog_param.sqlite3') # путь к рабочей базе данных экзогенных параметров SQLite

# константы базы данных SQLite3
strDatas_table='datas' # название таблицы с данными в базе данных SQLite
strHeaders_table='headers' # название таблицы с текстовыми и кодовыми именами рядов в базе данных SQLite


#pd.set_option('display.max_rows',None)

In [416]:
# выбрать переменные по кодам, которые должны использоваться в текущем блоке

# общие для всех блоков фактические ряды
list_Year=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
           'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x','HHAv')
# фактические данные для зависимых переменных из блока "ЖИЛИЩНОЕ СТРОИТЕЛЬСТВО"
list_YearZS=('BldProcMKD','VvodyMKDprivate','ProjectsPrivate_x','VvodyIZD','VvodySocFB',
             'VvodySocRB','AvSqVv','Vvodyunits','VvodyIZDunits','ZhilFond','ZhilFondIZD',
             'ZhilFondunits','ZhilFondIZDunits')
# общие для всех блоков экзогенные переменные
list_ExYear=('GDP_x','GDP_Iq','CPIAv','KeyRate','Unmpl_s','Inc_x','DispInc_I','ZPN_I',
             'ZPR_I','Invest_Iq','Retail_Iq','Activ_x','ER','Urals','Pop_x')
# общие для всех блоков экзогенные параметры
list_ExParam=('p_ProbDef','p_LabProdOnCost','p_GovPolOnCost','ConstrPriceIZD_x',
              'p_MortgLifeAv_x','HHAv')


In [417]:
strSELECT_Year='''select datas.date, datas.value, headers.code2
                  from {datas_table} join {header_table}
                  on {datas_table}.code={header_table}.code
                  where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_Year)
strSELECT_YearZS='''select datas.date,datas.value,headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code 
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_YearZS)
strSELECT_ExYear='''select datas.date, datas.value, headers.code2 
                    from {datas_table} join {header_table} 
                    on {datas_table}.code={header_table}.code
                    where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExYear)
strSELECT_ExParam='''select datas.date,datas.value,headers.code2 
                     from {datas_table} join {header_table} 
                     on {datas_table}.code={header_table}.code 
                     where headers.code2 in {list_p}'''.format(datas_table=strDatas_table, header_table=strHeaders_table, list_p=list_ExParam)



conWorkDB=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strYearDBPath)) # connection к рабочей базе данных фактических рядов 
conWorkEx=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExYearDBPath)) # connection к рабочей базе данных экзогенных переменных
conWorkExParam=sa.create_engine('sqlite+pysqlite:///{db_name}'.format(db_name=strExParamDBPath)) # connection к рабочей базе данных экзогенных параметров

pdfYear=pd.read_sql(strSELECT_Year, con=conWorkDB)
pdfYearZS=pd.read_sql(strSELECT_YearZS, con=conWorkDB)
pdfExYear=pd.read_sql(strSELECT_ExYear, con=conWorkEx)
pdfExParam=pd.read_sql(strSELECT_ExParam, con=conWorkExParam)

In [413]:
idx=list(range(pdfYear['date'].min(),pdfYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfYearMain=pd.DataFrame(index=idx)
for i in pdfYear['code2'].unique():
    pdfYearMain[i]=pdfYear[pdfYear['code2']==i].set_index('date').drop(columns=['code2'])

pdfYearMain[['GDP_x','Inc_x']]*=1000
pdfYearMain[['GDP_Iq','CPIAv','KeyRate','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
pdfYearMain['Activ_x']/=1000
# pdfYearMain

In [351]:
idx=list(range(pdfYearZS['date'].min(),pdfYearZS['date'].max()+1)) # задается период от первой до последней даты в БД (2004 - 2019 гг.)
pdfYearZSMain=pd.DataFrame(index=idx)
for i in pdfYearZS['code2'].unique():
    pdfZS[i]=pdfYearZS[pdfYearZS['code2']==i].set_index('date').drop(columns=['code2'])
# pdfZS

In [414]:
idx=list(range(pdfExYear['date'].min(),pdfExYear['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExYearMain=pd.DataFrame(index=idx)
for i in pdfExYear['code2'].unique():
    pdfExYearMain[i]=pdfExYear[pdfExYear['code2']==i].set_index('date').drop(columns=['code2'])
pdfExYearMain[['GDP_Iq','CPIAv','Unmpl_s','DispInc_I','ZPN_I','ZPR_I','Invest_Iq','Retail_Iq']]*=100
# pdfExYearMain

In [418]:
idx=list(range(pdfExParam['date'].min(),pdfExParam['date'].max()+1)) # задается период от первой до последней даты в БД  (2004 - 2019 гг.)
pdfExParamMain=pd.DataFrame(index=idx)
for i in pdfExParam['code2'].unique():
    pdfExParamMain[i]=pdfExParam[pdfExParam['code2']==i].set_index('date').drop(columns=['code2'])
pdfExParamMain

,p_ProbDef,p_MortgLifeAv_x,HHAv,ConstrPriceIZD_x,p_LabProdOnCost,p_GovPolOnCost
2007,0.10,NaN,NaN,NaN,NaN,NaN
2008,0.10,17.941667,NaN,NaN,NaN,NaN
2009,0.10,16.458333,NaN,NaN,NaN,NaN
2010,0.10,16.358333,NaN,NaN,NaN,NaN
2011,0.10,14.908333,NaN,NaN,NaN,NaN
2012,0.10,14.958333,NaN,NaN,NaN,NaN
2013,0.10,14.700000,NaN,NaN,NaN,NaN
2014,0.10,14.958333,NaN,NaN,NaN,NaN
2015,0.10,14.708333,2.450000,NaN,NaN,NaN
2016,0.10,15.250000,NaN,NaN,NaN,NaN


In [432]:
# Датафреймы должны соединяться по принципу: там где заканчивается фактический ряд - должны пристыковываться экзогенные данные. !!Нельзя обрезать весь фрейм экзогенных данных
# и просто мерджить/конкатенейтить/аппендить, так как один (или несколько) фактический ряд может быть заполнен не до конца. Например последняя точка в фактической БД будет 2017 г. 
# Тогда продолжение ряда должно будет браться из экзогенно-задаваемых значений, начиная с 2018 г.  

# Глупая идея: на первом шаге к фрейму фактических данных по его индексам будет мерджиться (inner) фрейм экзогенных (заполняя тем самым пропуски в конце факта, если они есть),
# а на втором шаге можно уже просто аппендить обрезанный экзог

# Попытки объединить pdfYearMain и pdfExYearMain за один шаг:
# pd.merge(pdfYearMain,pdfExYearMain,how='outer',left_index=True,right_index=True)
# pd.concat([pdfYearMain,pdfExYearMain], ignore_index=False, sort=False)
# pdfYearMain.append(pdfExYearMain,ignore_index=False, sort=False)
# pdfYearMain
# pd.merge(pdfYearMain,pdfExYearMain,how='inner',left_index=True,right_index=True)


# Пока просто аппенд, так не должно быть
PD=pdfYearMain.append(pdfExYearMain.loc[2020:2030])

# Еще нужно добавить параметр HHAv на прогнозном периоде. Но fillna не срабатывает с DataFrame, хотя тут говорится, что можно: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html
# PD['HHAv']=PD['HHAv'].fillna(pdfExParam[pdfExParam['code2']=='HHAv'].set_index('date').drop(columns=['code2']))
PD

,Pop_x,HHAv,GDP_x,GDP_Iq,Invest_Iq,Retail_Iq,ZPN_I,ZPR_I,Inc_x,DispInc_I,Unmpl_s,CPIAv,KeyRate,ER,Urals,Activ_x
2004,144.067300,2.677054,17027.190860,107.175949,116.800000,113.300000,122.569792,110.6000,6410.300000,110.400000,7.714864,110.881240,13.520000,28.817116,34.506738,7.137613
2005,143.518814,2.661470,21609.765489,106.376187,110.200000,112.800000,126.928783,112.6000,8088.300000,112.400000,7.153243,112.705532,12.980000,28.285366,50.775063,9.750847
2006,143.049637,2.645886,26917.201375,108.153432,117.800000,114.100000,124.301578,113.3000,10154.800000,113.500000,7.062219,109.683005,8.894216,27.191329,61.277787,14.046610
2007,142.805114,2.630302,33247.513229,108.535080,123.800000,116.100000,127.825842,117.2000,12540.200000,112.100000,6.029393,108.988092,7.512557,25.580734,69.659016,20.125188
2008,142.742366,2.614717,41276.849187,105.247954,109.500000,113.700000,127.197822,111.5000,14863.600000,102.400000,6.241192,114.104815,8.129107,24.854437,94.325000,28.022340
2009,142.785348,2.599133,38807.218575,92.179115,86.500000,94.900000,107.796414,96.5000,16895.000000,103.000000,8.252236,111.697689,11.310000,31.748954,60.891667,29.430000
2010,142.849468,2.583549,46308.541190,104.503726,106.300000,106.500000,112.415495,105.2000,18958.400000,105.900000,7.344543,106.850786,8.030000,30.369634,77.568171,33.804600
2011,142.960908,2.544156,60114.000837,104.264177,110.800000,107.100000,111.535892,102.8000,20780.000000,100.500000,6.495643,108.463393,8.120000,29.386460,109.325833,41.627500
2012,143.201721,2.504763,68103.449628,104.024086,106.800000,106.300000,113.950105,108.4000,23221.100000,104.600000,5.458664,105.066725,8.070000,31.089027,110.597506,49.509600
2013,143.506995,2.465370,72985.701130,101.755422,100.800000,103.900000,111.878028,104.8000,25684.400000,104.000000,5.478040,106.760812,7.350000,31.838413,108.051151,57.423100


## Дальше старые куски кода

Загрузка данных листа "Параметры" (27:26)

In [211]:
ddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=0,header=5,index_col=0)
ddf=ddf.loc[ddf.index.dropna()]
ddf=ddf.T

Загрузка данных листа "Жилищное строительство" (36:27)

In [206]:
hddf=pd.read_excel(r'../DB/SRC/Модель развития ЖС_краткая версия_ИТОГ_Целевой_с новой моделью - новая доля продаж.xlsx',sheet_name=3,header=5,index_col=0,nrows=45)
hddf=hddf.loc[hddf.index.dropna()]
hddf=hddf.iloc[:,1:-2]
#hddf.drop(hddf.columns[0],axis=1,inplace=True)
hddf=hddf.T

<div style="background:beige"><h3> Определение основных дат</h3>

  - `iLastFactYEAR` - последний год фактических данных. 2019
  - `iLastFORCAST` - год горизонта прогноза. 2030

    Пока задаются вручную
</div>

In [207]:
# iLastFactYEAR
iLastFactYEAR=conWork.connect().execute('select max(date) from datas').fetchone()[0]
iLastFORCAST=iLastFactYEAR+11

print('Последний фактический год - ', iLastFactYEAR, '; горизонт прогноза - ', iLastFORCAST)

Последний фактический год -  2019 ; горизонт прогноза -  2030


In [208]:
hddf.index=hddf.index.astype(int)
hddf_XY=hddf.truncate(after=iLastFactYEAR)
hddf_xy=hddf.truncate(before=iLastFactYEAR+1, after=iLastFORCAST)

In [210]:
hddf_XY

,Макроэкономические и демографические факторы,Темп прироста физического объема ВВП,"ИПЦ, в среднем за год",Уровень безработицы,Изменение реальных располагаемых доходов населения,"Численность населения, в среднем за год",Средний размер домохозяйства,Параметры данного блока,"Совокупный ввод жилья в рамках Национального проекта ""Жилье и городская среда""","Совокупный ввод жилья в МКД в рамках Национального проекта ""Жилье и городская среда""",...,Целевая плотность застройки в двадцати крупных городских агломерациях,Целевая плотность застройки в промышленных зонах городских центров двадцати крупных городских агломераций,Целевая плотность застройки на освоенных новых территориях в двадцати крупных городских агломерациях,Доля площади застраиваемых промышленных зон в черте двадцати центров крупных городских агломераций,Потенциал строительства жилья во всех промышленных зонах в черте двадцати центров крупных городских агломерациях,Коэффициент повышения плотности строительства в промышленных зонах,Параметры из других блоков модели,Средняя фактическая удельная стоимость строительства МКД,"Средняя цена 1 кв.м, приобретаемого по ДДУ, в МКД",Средняя цена 1 кв. м на первичном рынке
2005,NaN,6.37619,1.12706,0.0715324,1.124,143.519,2.66147,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.812,21.1567,23.7716
2006,NaN,8.15343,1.09683,0.0706222,1.135,143.05,2.64589,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.84,28.0115,31.4736
2007,NaN,8.53508,1.08988,0.0602939,1.121,142.805,2.6303,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.72,39.056,43.8832
2008,NaN,5.25646,1.14105,0.0624119,1.024,142.742,2.61472,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.622,45.6864,51.333
2009,NaN,-7.82474,1.11698,0.0825224,1.03,142.785,2.59913,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.312,43.4902,48.8654
2010,NaN,18.3923,1.06851,0.0734454,1.059,142.849,2.58355,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.877,42.8293,48.1228
2011,NaN,0.467491,1.08463,0.0649564,1.005,142.961,2.54416,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.32,37.8975,42.5814
2012,NaN,4.0176,1.05067,0.0545866,1.046,143.202,2.50476,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.354,41.5987,46.7401
2013,NaN,1.75175,1.06761,0.0547804,1.04,143.507,2.46537,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36.439,44.1202,49.5733
2014,NaN,0.744972,1.07804,0.0515632,0.988,146.091,2.42598,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,39.447,45.3205,50.9219


## Запас Жилья строящихся МКД

In [243]:
pdfNvstr['Тренд'] = None

In [256]:
pdfNvstr

,value,Тренд
date,,
2004,59.386684,63.125021
2005,66.863359,63.125021
2006,70.858939,68.861149
2007,68.941974,69.900456
2008,64.672504,66.807239
2009,68.476586,66.574545
2010,73.837691,71.157139
2011,81.098644,77.468168
2012,90.318934,85.708789


In [255]:
for i, row in pdfNvstr.iterrows():
    print(row.name)
    pdfNvstr['Тренд'][row.name+1]=np.mean([pdfNvstr['value'][row.name],pdfNvstr['value'][row.name+1]])

2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019


KeyError: 2020